<a href="https://colab.research.google.com/github/NitinMore9920/Credit_risk_modaling_machine_learning_project/blob/main/Credit_Risk_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit Risk Modeling using Machine Learning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, r2_score, precision_score, precision_recall_fscore_support

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
df1 = pd.read_excel("/content/drive/My Drive/final_project/Credit_Risk_Modeling_using_Machine_Learning/case_study1.xlsx")
df2 = pd.read_excel("/content/drive/My Drive/final_project/Credit_Risk_Modeling_using_Machine_Learning/case_study2.xlsx")

In [ ]:
print(df1.head(5))

   PROSPECTID  Total_TL  Tot_Closed_TL  Tot_Active_TL  Total_TL_opened_L6M  \
0           1         5              4              1                    0   
1           2         1              0              1                    0   
2           3         8              0              8                    1   
3           4         1              0              1                    1   
4           5         3              2              1                    0   

   Tot_TL_closed_L6M  pct_tl_open_L6M  pct_tl_closed_L6M  pct_active_tl  \
0                  0            0.000                0.0          0.200   
1                  0            0.000                0.0          1.000   
2                  0            0.125                0.0          1.000   
3                  0            1.000                0.0          1.000   
4                  0            0.000                0.0          0.333   

   pct_closed_tl  ...  CC_TL  Consumer_TL  Gold_TL  Home_TL  PL_TL  \
0         

### Remove null values

In [ ]:
df1.shape

(51336, 26)

In [ ]:
df1 = df1.loc[df1['Age_Oldest_TL'] != -99999]

In [ ]:
df1.shape

(51296, 26)

In [ ]:
df2.head(10)

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.000,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.000,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.000,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.000,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.000,0.0,-99999.000,0,0,AL,AL,753,P1
5,6,245,27,18,14,300,270,0,0,0,...,0.0,0.429,0.0,-99999.000,1,0,ConsumerLoan,PL,668,P3
6,7,49,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.000,0.0,-99999.000,1,0,others,others,703,P1
7,8,74,23,12,3,164,133,0,0,0,...,0.0,0.000,0.0,-99999.000,0,0,ConsumerLoan,others,676,P2
8,9,424,7,4,3,99,38,2,3,1,...,0.0,0.000,0.0,-99999.000,0,0,ConsumerLoan,others,658,P4
9,10,39,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.000,0.0,3.333,0,0,others,others,705,P1


In [ ]:
column_to_be_remove = []
for i in df2.columns:
  if df2.loc[df2[i] == -99999].shape[0]>10000:
    column_to_be_remove.append(i)

In [ ]:
df2 = df2.drop(column_to_be_remove, axis=1)

In [ ]:
df2.shape

(51336, 54)

In [ ]:
df2 = df2.replace(-99999, np.nan)
df2 = df2.dropna()

df2.isna().sum()

,0
PROSPECTID,0
time_since_recent_payment,0
num_times_delinquent,0
max_recent_level_of_deliq,0
num_deliq_6mts,0
num_deliq_12mts,0
num_deliq_6_12mts,0
num_times_30p_dpd,0
num_times_60p_dpd,0
num_std,0


In [ ]:
df2.shape

(42066, 54)

In [ ]:
# Checking comons column names

for i in list(df1.columns):
  if i in list(df2.columns):
    print(i)

PROSPECTID


In [ ]:
# Mearge two dataframes, inner join so that no null values are present in our dataset
df = pd.merge(df1,df2,how='inner', left_on=['PROSPECTID'], right_on=['PROSPECTID'])

In [ ]:
df.head(2)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.0,0.0,0.2,0.8,...,0.0,0.0,0.0,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2


In [ ]:
df.shape

(42064, 79)

In [ ]:
df.isnull().sum()

,0
PROSPECTID,0
Total_TL,0
Tot_Closed_TL,0
Tot_Active_TL,0
Total_TL_opened_L6M,0
...,...
GL_Flag,0
last_prod_enq2,0
first_prod_enq2,0
Credit_Score,0


In [ ]:
# first split dataset
X = df.drop('Approved_Flag', axis=1)
y = df['Approved_Flag']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
# Check the how many columns are categorical

for i in df.columns:
  if df[i].dtype == 'object':
    print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(33651, 78)
(8413, 78)
(33651,)
(8413,)


In [ ]:
# Chi-square ONLY on training data

from scipy.stats import chi2_contingency

categorical_cols = [
    'MARITALSTATUS',
    'EDUCATION',
    'GENDER',
    'last_prod_enq2',
    'first_prod_enq2'
]

chi_square_results = {}

for col in categorical_cols:
    contingency_table = pd.crosstab(X_train[col], y_train)
    chi2, pval, _, _ = chi2_contingency(contingency_table)

    chi_square_results[col] = pval
    print(f"{col}: p-value = {pval}")

MARITALSTATUS: p-value = 2.3000478188111667e-184
EDUCATION: p-value = 3.6697025024361895e-24
GENDER: p-value = 7.948693899293114e-05
last_prod_enq2: p-value = 0.0
first_prod_enq2: p-value = 1.4207969390149238e-223


In [ ]:
# since all the categorical featurs have pval <= 0.05 , we will accept all

In [ ]:
def calculate_vif(df, threshold=6, min_features=2):
    vif_df = df.copy()

    while vif_df.shape[1] > min_features:
        vif = pd.DataFrame()
        vif["feature"] = vif_df.columns
        vif["VIF"] = [
            variance_inflation_factor(vif_df.values, i)
            for i in range(vif_df.shape[1])
        ]

        vif["VIF"] = vif["VIF"].replace([np.inf, -np.inf], 999)

        max_vif = vif["VIF"].max()

        if max_vif > threshold:
            drop_feature = vif.sort_values("VIF", ascending=False)["feature"].iloc[0]
            print(f"Dropping {drop_feature} | VIF = {max_vif}")
            vif_df = vif_df.drop(columns=[drop_feature])
        else:
            break

    return vif_df.columns.tolist()


In [ ]:

numerical_cols = [
    col for col in X_train.columns
    if X_train[col].dtype != 'object' and col != 'PROSPECTID'
]


X_train_num = X_train[numerical_cols].copy()
X_train_num = X_train_num.replace([np.inf, -np.inf], np.nan)
X_train_num = X_train_num.dropna()

numerical_columns_after_vif = calculate_vif(
    X_train_num,
    threshold=6
)



/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Dropping pct_closed_tl | VIF = 1790.1811379960275


/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Dropping Total_TL | VIF = 999.0


/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


...

Dropping PL_TL | VIF = 999.0


/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Dropping num_deliq_12mts | VIF = 999.0


/usr/local/lib/python3.12/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Dropping pct_active_tl | VIF = 999.0
Dropping Secured_TL | VIF = 215.07837511051954
Dropping Credit_Score | VIF = 46.18484143283551
Dropping enq_L12m | VIF = 37.24104710619221
Dropping num_std_12mts | VIF = 26.526240102865156
Dropping pct_PL_enq_L6m_of_L12m | VIF = 24.71391847711679
Dropping Total_TL_opened_L12M | VIF = 23.02440082550371
Dropping pct_CC_enq_L6m_of_L12m | VIF = 19.47208202799184
Dropping enq_L6m | VIF = 16.720356247363853
Dropping AGE | VIF = 15.627298172746132
Dropping Unsecured_TL | VIF = 15.427659429604965
Dropping Tot_Active_TL | VIF = 14.297787883885743
Dropping num_times_30p_dpd | VIF = 13.779343488862946
Dropping PL_enq_L12m | VIF = 12.50018727700866
Dropping tot_enq | VIF = 10.012926948310918
Dropping Tot_TL_closed_L12M | VIF = 9.14595858356828
Dropping CC_enq_L12m | VIF = 9.008498406846181
Dropping num_dbt_12mts | VIF = 8.423273542625472
Dropping pct_tl_open_L6M | VIF = 7.254531644667829


In [ ]:
numerical_cols_after_vif = numerical_columns_after_vif
print(len(numerical_cols_after_vif))


48


In [ ]:
# check annova for columns_to_be_kept
X_train_num = X_train.loc[X_train_num.index, numerical_cols_after_vif]
y_train_aligned = y_train.loc[X_train_num.index]

from scipy.stats import f_oneway

anova_selected_features = []

for col in numerical_cols_after_vif:
    group_P1 = X_train_num[y_train_aligned == 'P1'][col]
    group_P2 = X_train_num[y_train_aligned == 'P2'][col]
    group_P3 = X_train_num[y_train_aligned == 'P3'][col]
    group_P4 = X_train_num[y_train_aligned == 'P4'][col]

    if min(len(group_P1), len(group_P2), len(group_P3), len(group_P4)) > 1:
        f_stat, p_val = f_oneway(group_P1, group_P2, group_P3, group_P4)

        if p_val <= 0.05:
            anova_selected_features.append(col)

        print(f"{col} → p-value = {p_val}")



Total_TL_opened_L6M → p-value = 1.6741172078978514e-158
Tot_TL_closed_L6M → p-value = 1.0422147011434219e-34
pct_tl_closed_L6M → p-value = 1.458816341805365e-38
pct_tl_open_L12M → p-value = 0.0
pct_tl_closed_L12M → p-value = 2.2585421086068175e-36
Tot_Missed_Pmnt → p-value = 4.475451211419432e-158
Auto_TL → p-value = 2.985221400267475e-225
CC_TL → p-value = 2.171324792790361e-47
Consumer_TL → p-value = 1.910251774984831e-24
Gold_TL → p-value = 6.077774982392837e-134
Home_TL → p-value = 3.4655935568638524e-299
Other_TL → p-value = 0.0
Age_Oldest_TL → p-value = 0.0
Age_Newest_TL → p-value = 7.809456182130138e-70
time_since_recent_payment → p-value = 9.214419426217623e-69
num_times_delinquent → p-value = 7.964402260922353e-61
max_recent_level_of_deliq → p-value = 1.1419419849388228e-66
num_deliq_6mts → p-value = 1.274477810819971e-14
num_deliq_6_12mts → p-value = 1.452263736320472e-43
num_times_60p_dpd → p-value = 6.258640040655614e-45
num_std → p-value = 0.0
num_std_6mts → p-value = 0.0


In [ ]:
#Final numerical features after ANOVA
print("Numerical features kept after ANOVA:")
print(len(anova_selected_features))


Numerical features kept after ANOVA:
43


In [ ]:
df.shape

(42064, 79)

In [ ]:
# feature selection is done for both categorical and numerical columns
# Lable encoding for the categorical features
# ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [ ]:
print(df['MARITALSTATUS'].unique()) # apply one hot encoding
print(df['EDUCATION'].unique())    # apply Lable encoding
print(df['GENDER'].unique())      # apply one hot encoding
print(df['last_prod_enq2'].unique())    # apply one hot encoding
print(df['first_prod_enq2'].unique())    # apply one hot encoding

['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
['M' 'F']
['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [ ]:
education_map = {
    'SSC': 1,
    '12TH': 2,
    'GRADUATE': 3,
    'UNDER GRADUATE': 3,
    'PROFESSIONAL': 3,
    'POST-GRADUATE': 4,
    'OTHERS': 1
}

df['EDUCATION'] = df['EDUCATION'].map(education_map).astype(int)


In [ ]:
df['EDUCATION'].value_counts()

,count
EDUCATION,
3,18931
2,11703
1,9532
4,1898


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [ ]:
df_encoded = pd.get_dummies(
    df,
    columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'],
    drop_first=True
)


In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 87 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    42064 non-null  int64  
 1   Total_TL                      42064 non-null  int64  
 2   Tot_Closed_TL                 42064 non-null  int64  
 3   Tot_Active_TL                 42064 non-null  int64  
 4   Total_TL_opened_L6M           42064 non-null  int64  
 5   Tot_TL_closed_L6M             42064 non-null  int64  
 6   pct_tl_open_L6M               42064 non-null  float64
 7   pct_tl_closed_L6M             42064 non-null  float64
 8   pct_active_tl                 42064 non-null  float64
 9   pct_closed_tl                 42064 non-null  float64
 10  Total_TL_opened_L12M          42064 non-null  int64  
 11  Tot_TL_closed_L12M            42064 non-null  int64  
 12  pct_tl_open_L12M              42064 non-null  float64
 13  p

In [ ]:
k = df_encoded.describe()
k

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_currentBal_all_TL,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,Credit_Score
count,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.00000,42064.000000,42064.000000,42064.000000,42064.000000,...,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000
mean,25649.827477,5.262980,2.967383,2.295597,0.812643,0.48992,0.179032,0.097783,0.577452,0.422548,...,0.883693,0.102962,0.193063,0.219169,0.074833,0.195497,0.064186,0.252235,0.056580,679.326336
std,14844.173396,7.463383,6.141098,2.404086,1.383559,1.05892,0.278043,0.210957,0.366110,0.366110,...,40.622275,0.303913,0.394707,0.395100,0.250658,0.367414,0.225989,0.434300,0.231042,21.133619
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,469.000000
25%,12776.750000,1.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.286000,0.000000,...,0.152000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,668.000000
50%,25706.500000,3.000000,1.000000,2.000000,0.000000,0.00000,0.000000,0.000000,0.545000,0.455000,...,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,679.000000
75%,38518.250000,6.000000,3.000000,3.000000,1.000000,1.00000,0.333000,0.100000,1.000000,0.714000,...,0.860000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,690.000000
max,51336.000000,235.000000,216.000000,47.000000,27.000000,19.00000,1.000000,1.000000,1.000000,1.000000,...,6327.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,809.000000


In [ ]:
# Encode TRAIN and TEST separately but consistently

X_train_enc = pd.get_dummies(
    X_train,
    columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'],
    drop_first=True
)

X_test_enc = pd.get_dummies(
    X_test,
    columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'],
    drop_first=True
)


In [ ]:
X_train_enc, X_test_enc = X_train_enc.align(
    X_test_enc, join='left', axis=1, fill_value=0
)


In [ ]:
# Keep ONLY selected numerical features (after VIF + ANOVA)

X_train_final = X_train_enc[anova_selected_features +
                            [col for col in X_train_enc.columns if col not in X_train.columns]]

X_test_final = X_test_enc[X_train_final.columns]


In [ ]:
print(len(X_train_final.columns))

55


# Machine Learing Model fitting
## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

rf_classifier.fit(X_train_final, y_train)


RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [ ]:
y_pred = rf_classifier.predict(X_test_final)


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7726138119576845


In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, _ = precision_recall_fscore_support(
    y_test,
    y_pred,
    average='weighted'
)

print(f"Precision: {precision}")
print(f"Recall   : {recall}")
print(f"F1-score : {f1}")


Precision: 0.7463553335593649
Recall   : 0.7726138119576845
F1-score : 0.7507540520947096


In [ ]:
precision_c, recall_c, f1_c, support = precision_recall_fscore_support(
    y_test,
    y_pred
)

for i, cls in enumerate(rf_classifier.classes_):
    print(f"\nClass {cls}")
    print(f"Precision: {precision_c[i]}")
    print(f"Recall   : {recall_c[i]}")
    print(f"F1-score : {f1_c[i]}")



Class P1
Precision: 0.8387096774193549
Recall   : 0.714867617107943
F1-score : 0.7718526663001649

Class P2
Precision: 0.802912292583813
Recall   : 0.9316306483300589
F1-score : 0.8624954528919607

Class P3
Precision: 0.44294294294294295
Recall   : 0.22903726708074534
F1-score : 0.30194472876151485

Class P4
Precision: 0.7579681274900398
Recall   : 0.7226970560303894
F1-score : 0.7399124939231891


In [ ]:
from xgboost import XGBClassifier


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

print(le.classes_)   # ['P1', 'P2', 'P3', 'P4']


['P1' 'P2' 'P3' 'P4']


In [ ]:
xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softprob',
    num_class=4,
    random_state=42,
    n_jobs=-1,
    eval_metric='mlogloss'
)


In [ ]:
xgb_model.fit(X_train_final, y_train_enc)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=-1, num_class=4, ...)

In [ ]:
y_pred_enc = xgb_model.predict(X_test_final)
y_pred = le.inverse_transform(y_pred_enc)


In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7824794960180673


In [ ]:
precision_c, recall_c, f1_c, support = precision_recall_fscore_support(
    y_test,
    y_pred
)

for i, cls in enumerate(le.classes_):
    print(f"\nClass {cls}")
    print(f"Precision: {precision_c[i]}")
    print(f"Recall   : {recall_c[i]}")
    print(f"F1-score : {f1_c[i]}")



Class P1
Precision: 0.8331515812431843
Recall   : 0.7780040733197556
F1-score : 0.8046340179041601

Class P2
Precision: 0.8302189001601709
Recall   : 0.9165029469548134
F1-score : 0.871229806704641

Class P3
Precision: 0.4433106575963719
Recall   : 0.30357142857142855
F1-score : 0.36036866359447006

Class P4
Precision: 0.7668341708542713
Recall   : 0.724596391263058
F1-score : 0.7451171875


In [ ]:
# Build the Decision Tree model
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(
    criterion='gini',
    max_depth=6,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42
)


In [ ]:
dt_model.fit(X_train_final, y_train)


DecisionTreeClassifier(max_depth=6, min_samples_leaf=10, min_samples_split=20,
                       random_state=42)

In [ ]:
y_pred_dt = dt_model.predict(X_test_final)

from sklearn.metrics import accuracy_score

accuracy_dt = accuracy_score(y_test, y_pred_dt)
print(f"Accuracy: {accuracy_dt}")


Accuracy: 0.7393319862118151


In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, _ = precision_recall_fscore_support(
    y_test,
    y_pred_dt,
    average='weighted'
)

print(f"Precision: {precision}")
print(f"Recall   : {recall}")
print(f"F1-score : {f1}")


Precision: 0.7074618733919219
Recall   : 0.7393319862118151
F1-score : 0.7132003436201904


In [ ]:
precision_c, recall_c, f1_c, support = precision_recall_fscore_support(
    y_test,
    y_pred_dt
)

for i, cls in enumerate(dt_model.classes_):
    print(f"\nClass {cls}")
    print(f"Precision: {precision_c[i]}")
    print(f"Recall   : {recall_c[i]}")
    print(f"F1-score : {f1_c[i]}")



Class P1
Precision: 0.7297592997811816
Recall   : 0.6792260692464358
F1-score : 0.7035864978902954

Class P2
Precision: 0.7905275988159498
Recall   : 0.8919449901768173
F1-score : 0.8381796362965014

Class P3
Precision: 0.4102079395085066
Recall   : 0.16847826086956522
F1-score : 0.23885525591634563

Class P4
Precision: 0.6487367563162184
Recall   : 0.7559354226020892
F1-score : 0.6982456140350877


Accuraci values

Random Forest: 0.77

Xgboost: 0.78

Decision tree : 0.74

Accuracy value for Xgboost is better. now further finetune it.

Hp tunning

Feature Engineering - Scaling, Feature engineeering, Graphs

In [ ]:
# Hyperparameter tunning for Xgboost model

# Define Hyperparameter Grid

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 8, 10],
    'learning_rate': [ 0.01, 0.1, 1],
    'alpha': [1, 10, 100],
    'colsample_bytree': [ 0.5, 0.7, 0.9, 1.1]
}


In [ ]:
# Define base XGBoost model

from xgboost import XGBClassifier

xgb_base = XGBClassifier(
    objective='multi:softprob',
    num_class=4,
    eval_metric='mlogloss',
    random_state=42,
    n_jobs=-1
)


In [61]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    scoring='f1_weighted',   # IMPORTANT for imbalanced multiclass
    cv=3,                    # keep it light
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train_final, y_train_enc)


Fitting 3 folds for each of 432 candidates, totalling 1296 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
324 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
324 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/xgboost/core.py", line 774, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/xgboost/sklearn.py", line 1808, in fit
    self._Booster = train(
                    ^^^^^^
  File "/usr/local/lib/python3.1

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     feature_weights=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constrain...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=-1, num_class=4, ...),
             n_jobs=-1,
             param_grid={'alpha': [1, 10, 100],
                         'colsample_bytree': [0.5, 0.7, 0.9, 1.1],
                         'learning_rate': [0.01, 0.1, 1],
                         'max_depth': [3, 5, 8, 10],
                         'n_estimators': [10, 50, 100]},
             scoring='f1_weighted', verbose=1)

In [62]:
print("Best parameters found:")
print(grid_search.best_params_)

best_xgb_model = grid_search.best_estimator_


Best parameters found:
{'alpha': 10, 'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 100}


In [63]:
# Evaluate tuned XGBoost on TEST data

y_pred_enc = best_xgb_model.predict(X_test_final)
y_pred = le.inverse_transform(y_pred_enc)





In [64]:
# Accuracy

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7804588137406395


In [65]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, _ = precision_recall_fscore_support(
    y_test,
    y_pred,
    average='weighted'
)

print(f"Precision: {precision}")
print(f"Recall   : {recall}")
print(f"F1-score : {f1}")


Precision: 0.761032213395652
Recall   : 0.7804588137406395
F1-score : 0.7671091423356158


In [66]:
# Save the trained model

import joblib

# XGBoost model
joblib.dump(best_xgb_model, "xgboost_credit_risk_model.pkl")

# label encoder (VERY IMPORTANT)
joblib.dump(le, "label_encoder.pkl")

# preprocessing objects
joblib.dump(X_train_final.columns.tolist(), "model_features.pkl")



['model_features.pkl']

In [67]:
# Load model for inference (deployment use)

import joblib

# Load everything
model = joblib.load("xgboost_credit_risk_model.pkl")
label_encoder = joblib.load("label_encoder.pkl")
model_features = joblib.load("model_features.pkl")


In [68]:
# Predict on NEW data

# new_data: pandas DataFrame with raw features
new_data = new_data[model_features]  # enforce same column order

pred_enc = model.predict(new_data)
pred_label = label_encoder.inverse_transform(pred_enc)

print(pred_label)
